# QTNM Update 11/11/21

## Radiated power of a single electron

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
from scipy.constants import c, epsilon_0 as ep0, elementary_charge as qe, electron_mass as me, mu_0 as mu0
from SingleElectronRadiation import SingleElectronRadiation as SER

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
# Credit: https://stackoverflow.com/questions/38194247/how-can-i-connect-two-points-in-3d-scatter-plot-with-arrow
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [ ]:
fig = plt.figure(figsize=[8,8])
ax = plt.axes(projection='3d')
ax.set_aspect('auto')

# Draw centered axes
val = [1,0,0]
labels = ['x', 'y', 'z']
colors = ['black', 'black', 'black']
for v in range(3):
    x = [val[v-0], -val[v-0]]
    y = [val[v-1], -val[v-1]]
    z = [val[v-2], -val[v-2]]
    ax.plot(x,y,z,'k-', linewidth=1)
    f = 1.075
    ax.text(f*val[v-0], f*val[v-1], f*val[v-2], labels[v], color=colors[v], fontsize=20)


# Hide everything else
# Hide axes ticks
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
# make the panes transparent
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# Hide box axes
ax._axis3don = False

# Expand to remove white space
lim = 0.65
ax.set_xlim(np.array([-1,1])*lim)
ax.set_ylim(np.array([-1,1])*lim)
ax.set_zlim(np.array([-1,1])*lim)

# Plot some vectors
_beta_dot = [0.75, 0, 0]
_beta = [0, 0, 0.75]
_n = [0.25, 0.75, 0.95]

arw = Arrow3D([0,_beta_dot[0]],[0,_beta_dot[1]],[0,_beta_dot[2]], arrowstyle="->", color="red",
              lw = 4, mutation_scale=25, alpha=0.5)
ax.add_artist(arw)
ax.text(_beta_dot[0],0.1+_beta_dot[1],_beta_dot[2], r'$\dot{\vec{\beta}}$', color="red", fontsize=20)

arw = Arrow3D([0,_beta[0]],[0,_beta[1]],[0,_beta[2]], arrowstyle="->", color="red",
              lw = 4, mutation_scale=25, alpha=0.5)
ax.add_artist(arw)
ax.text(_beta[0],0.05+_beta[1],_beta[2], r'$\vec{\beta}$', color="red", fontsize=20)

arw = Arrow3D([0,_n[0]],[0,_n[1]],[0,_n[2]], arrowstyle="->", color="blue",
              lw = 4, mutation_scale=25, alpha=0.5)
ax.add_artist(arw)
ax.text(_n[0],_n[1],_n[2], r'$\vec{n}$', color="blue", fontsize=20)

plt.plot([0,_n[0]],[0,_n[1]],[0,0], lw=4, alpha=0.5, color='blue', linestyle='--')
plt.plot([_n[0],_n[0]],[_n[1],_n[1]],[0,_n[2]], lw=4, alpha=0.5, color='blue', linestyle='--')

# Want to draw an arc from beta_dot / 2 -> [_n[0], _n[1], 0]
radius = _beta_dot[0] / 2
tmax = np.arctan(_n[1] / _n[0])
theta = np.linspace(0, tmax, 11)
xs = radius * np.cos(theta)
ys = radius * np.sin(theta)
zs = np.zeros_like(xs)
plt.plot(xs, ys, zs, color='black')
ax.text(1.4*xs[3], 1.4*ys[3], zs[3], r'$\phi$', fontsize=20)

# This is not actually the correct arc (wrong plane), but good enough for our purposes
radius = _beta[2] / 2
tmax = np.arctan(_n[1] / _n[2])
theta = np.linspace(0, tmax, 11)
zs = radius * np.cos(theta)
ys = radius * np.sin(theta)
xs = np.zeros_like(xs)
plt.plot(xs, ys, zs, color='black')
ax.text(xs[3], ys[3], 1.1*zs[3], r'$\theta$', fontsize=20)

ax.view_init(30, 30)

plt.tight_layout()
plt.title('3D Coordinate System', fontsize=20)
plt.show()

# Set up a detector on the z-axis, at a distance of 2cm from the origin

In [ ]:
R = 2e-2 # Radius for antenna
antennaPosition = [0,0,R] # Dipole on z axis aligned in x direction
antennaAlignment = [1,0,0] # Antenna alignment

In [ ]:
# Some routines to specify the electron's path

def positions(times, radius=4.6e-4, f=2.7e10):
    return rl * np.cos(2.0 * np.pi * times * f), np.zeros_like(times), rl * np.sin(2.0 * np.pi * times * f)

def velocities(times, radius=4.6e-4, f=2.7e10):
    v0 = rl * f * 2.0 * np.pi
    return -v0 * np.sin(2.0 * np.pi * times * f), np.zeros_like(times), v0 * np.cos(2.0 * np.pi * times * f)

def accelerations(times, radius=4.6e-4, f=2.7e10):
    a0 = rl * (f * 2.0 * np.pi)**2
    return -a0 * np.cos(2.0 * np.pi * times * f), np.zeros_like(times), -a0 * np.sin(2.0 * np.pi * times * f)

In [ ]:
B = 1.0 # Tesla

# Calculate frequency, Larmor radius etc properly
# Initial kinetic energy (eV)
T = 18600
# Rel. gamma
gamma = T * qe / (me*c**2) + 1
# (v/c)^2
beta = np.sqrt(1 - 1 / gamma**2)

# Calculate frequency, Larmor radius
rl = gamma * me * beta * c / (qe * B)
f = qe * B / (2.0 * np.pi * gamma * me)

wl = c / f # Wavelength

# Effective area of Hertzian dipole
ae = wl**2 * 3 / (8 * np.pi)

## Our antenna will be modelled as a Hertzian dipole, with effective area of:

\begin{equation*}
A_e = \frac{3 \lambda^2}{8 \pi}
\end{equation*}

In [ ]:
print('Antenna effective area = %.6e' % ae)

print('Electron Orbit parameters:')

print('Magnetic field =  %.2f T' % B)
print('Electron kinetic energy = %.2f KeV' % (T/1e3))
print('gamma, beta = %.6f, %.6f' % (gamma, beta))
print('Larmor radius = %.6f mm' % (rl*1e3))
print('Frequency = %.4f GHz' % (f/1e9))

## The angular distribution of power is:

\begin{align*}
\frac{d P(t')}{d\Omega} &= \left(\frac{e a}{4 \pi \epsilon_0 c^2 R} \right)^2 \frac{1}{\mu_0 c} \frac{\left|\vec{n} \times \left[\left(\vec{n} - \vec{\beta} \right) \times\dot{\vec{\beta}} \right] \right|^2}{\left(1 - \vec{n}\cdot\vec{\beta} \right)^5} \\
&= \left(\frac{e a}{4 \pi \epsilon_0 c^2 R} \right)^2 \frac{1}{\mu_0 c}\left[\left(\cos \theta - \beta\right)^2 + \sin^2\phi \sin^2 \theta \left(1 - \beta^2\right) \right] \frac{1}{(1 - \beta \cos \theta)^5}
\end{align*}

## Here $a$ is the magnitude of the acceleration (constant). This is equivalent to equation (14.44) of Jackson and Stafford's notes. Written this way as about to set $\phi = 0$

## For $\phi = 0$ this has a maximum at $\theta = 0$. Using our effective area from before:

In [ ]:
# Set up solution

# Set of times
times = np.linspace(0.0, 4.0 / f, 4001)

x, y, z = positions(times, radius=rl, f=f)
vx, vy, vz = velocities(times, radius=rl, f=f)
ax, ay, az = accelerations(times, radius=rl, f=f)

a0 = np.linalg.norm(accelerations(0, radius=rl, f=f)) # Need this in a bit. Constant here

In [ ]:
max_value = (a0 * qe / (4.0 * np.pi * ep0 * c**2 * R))**2 / (mu0 * c) / (1.0 - beta)**3
print('Maximum power = %.4E' % (max_value * ae))

In [ ]:
# Use Daniel's code to work out the relativistic fields and Poynting magnitude
poyntingMagnitude = np.zeros(len(times))

# Work out far field according to Daniel's code
for i in range(len(times)):
    # Electron circular motion in xy plane
    ePosX = x[i]
    ePosY = y[i]
    ePosZ = z[i]
    
    eVelX = vx[i]
    eVelY = vy[i]
    eVelZ = vz[i]
    
    eAccX = ax[i]
    eAccY = ay[i]
    eAccZ = az[i]
    
    EPosition = np.array([ePosX,ePosY,ePosZ])
    EVelocity = np.array([eVelX,eVelY,eVelZ])
    EAcceleration = np.array([eAccX,eAccY,eAccZ])
    
    # Only interested in far field
    farEField = SER.CalcRelFarEField(antennaPosition,times[i],EPosition,EVelocity,EAcceleration)[0]
    
    poyntingMagnitude[i] = SER.CalcPoyntingVectorMagnitude(np.linalg.norm(farEField))

In [ ]:
# Now work out theoretical power according to Jackson formula
def theory_electron(theta, a0, beta, R):
    fac = qe / (4.0 * np.pi * ep0 * c**2 * R)
    smag = fac**2 * a0**2 / mu0 / c
    return smag * (np.cos(theta) - beta)**2 / (1.0 - beta * np.cos(theta))**5

theta = 2.0 * np.pi * f * times

In [ ]:
plt.plot(theta, theory_electron(theta, a0, beta, R), label=r'Jackson (14.44)')
plt.plot(theta, poyntingMagnitude, linestyle='--', label='Daniel\'s code')
plt.xlim(left=0.0)
plt.ylim(bottom=0)
plt.xlabel('Angle (radians)')
plt.ylabel(r'Power $(W/m^2)$')
plt.xlim(left=0.0)
plt.ylim(bottom=0)
plt.legend()
plt.tight_layout()

## This doesn't match. The previous equation (e.g 14.38 or 14.44 of Jackson) is power radiated in terms of the electron time, $t'$. Need to multiply by

\begin{equation*}
\frac{dt'}{dt} = \frac{1}{1 - \vec{\beta} \cdot \vec{n}} = \frac{1}{1 - \beta \cos \theta}
\end{equation*}

(See e.g. 14.35-14.37 of Jackson)

In [ ]:
def theory_lab(theta, a0, beta, R):
    return theory_electron(theta, a0, beta, R) / (1.0 - beta * np.cos(theta))

In [ ]:
plt.plot(theta, theory_lab(theta, a0, beta, R), label=r'$Jackson (14.44)  / (1.0 - \beta\cos\theta)$')
plt.plot(theta, poyntingMagnitude, linestyle='--', label='Daniel\'s code')
plt.xlim(left=0.0)
plt.ylim(bottom=0)
plt.xlabel('Angle (radians)')
plt.ylabel(r'Power $(W/m^2)$')
plt.xlim(left=0.0)
plt.ylim(bottom=0)
plt.legend()
plt.tight_layout()

## Matches a lot better. Note that the theory isn't totally accurate - assumes that distance = antenna position, rather than electron position - antetenna position. Consequently some small errors in theta too.

## Finally multiply by effective area of our dipole, to get the power received by the antenna

In [ ]:
powersHertzian = np.zeros(len(times))

# Work out far field according to Daniel's code
for i in range(len(times)):
    # Electron circular motion in xy plane
    ePosX = x[i]
    ePosY = y[i]
    ePosZ = z[i]
    
    eVelX = vx[i]
    eVelY = vy[i]
    eVelZ = vz[i]
    
    eAccX = ax[i]
    eAccY = ay[i]
    eAccZ = az[i]
    
    EPosition = np.array([ePosX,ePosY,ePosZ])
    EVelocity = np.array([eVelX,eVelY,eVelZ])
    EAcceleration = np.array([eAccX,eAccY,eAccZ])
    
    # Only interested in far field
    farEField = SER.CalcRelFarEField(antennaPosition,times[i],EPosition,EVelocity,EAcceleration)[0]
    
    _poyntingMagnitude = SER.CalcPoyntingVectorMagnitude(np.linalg.norm(farEField))
    
    
    antennaToElectronVector = EPosition - antennaPosition
    cosDipoleToEmissionAngle = np.dot(antennaToElectronVector,antennaAlignment)  \
    / ( np.linalg.norm(antennaToElectronVector)*np.linalg.norm(antennaAlignment) )
    
    dipoleToEmissionAngle = np.arccos(cosDipoleToEmissionAngle)
    hertzianDipoleEffectiveArea = SER.HertzianDipoleEffectiveArea(wl,dipoleToEmissionAngle)
    
    powersHertzian[i] = _poyntingMagnitude * hertzianDipoleEffectiveArea

In [ ]:
plt.plot(theta, theory_lab(theta, a0, beta, R) * ae, 
         label=r'$Jackson (14.44)  / (1.0 - \beta\cos\theta) * A_e$')
plt.plot(theta, powersHertzian, linestyle='--', label='Daniel\'s code')
plt.xlim(left=0.0)
plt.ylim(bottom=0)
plt.xlabel('Angle (radians)')
plt.ylabel('Power (W)')
plt.xlim(left=0.0)
plt.ylim(bottom=0)
plt.legend()
plt.tight_layout()

In [ ]:
max_value = (a0 * qe / (4.0 * np.pi * ep0 * c**2 * R))**2 / (mu0 * c) / (1.0 - beta)**4
print('Maximum power (defined in lab time) = %.4E' % (max_value * ae))

## Summary

 - By adjusting textbook result to lab time (multiply by $dt' / dt$) can obtain good match between text book result and existing radiation code.
 - Results shown here are far-field only.
 - Peak power received by Hertzian dipole 2cm from electron is $\sim 0.015 fW$, which is still lower the $\sim 0.028 fW$ reported by Stafford.